<a href="https://colab.research.google.com/github/manggalih/2211102269-W10/blob/master/OLAH_DATA_FINAL1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TAHAP 1 DATA PREPARATION**

LANGKAH 1 IMPORT LIBRARY

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder


LANGKAH 2 LOAD DATASET FINAL

In [2]:
from google.colab import files
uploaded = files.upload()


Saving DATA_FINAL.xlsx to DATA_FINAL.xlsx


In [3]:
import pandas as pd

df = pd.read_excel("DATA_FINAL.xlsx")

print("Jumlah data dan kolom:", df.shape)
df.head()


Jumlah data dan kolom: (339, 15)


,No,Nama Pasien,Jenis Kelamin,Tanggal Lahir,Alamat,Nomor HP,Tanggal Pemeriksaan,Pemeriksaan,Diagnosa/Hasil Pemeriksaan,Pengobatan/Resep,Total Harga Pembayaran,Tekanan Darah,Suhu,Berat Badan,Tinggi Badan
0,1,Tn. Abdul Kholiq,Laki-laki,1971-10-12 00:00:00,kdw 01/05,082241846417,03/01/2023,"Keadaan Umum: Baik, td ada keluhan, Riwayat HT...",SKD (surat keterangan Dokter),0,35000.0,100/80,0,63,0.0
1,2,Ny. Algis Bagil,Perempuan,03/08/1999,Karang Tengah 09/01,082136710103,02/10/2023,"ambeyen keluar darah terus saat BAB, saat ini ...",Hemoroid Interna,"Asam tranexsamat 2x1 tablet, Vit C 3x2 tablet",51000.0,100/79,0,64,0.0
2,3,Tn. Akbar Vicky H,Laki-laki,23/05/1998,Perum. Kedungwringin blok c no. 69,081515949679,20/05/2023,"Demam, batuk berdahak, pilek, nyeri tenggorokan",ISPA,"Flucadex 3x1 tablet, dexametasone 3x1 tablet, ...",55000.0,130/80,38,0,0.0
3,4,An. Arreta Azkayra,Perempuan,05/11/2011,kdw 01/03,0,04/11/2023,"Demam sejak 4hari yang lalu, batuk pilek, pusi...","observasi febris H+4, ISPA, DISPESIA","Paracetamol 3x1 tablet, sefatroksil 2x1 tablet...",56000.0,0,37.8,36,0.0
4,5,Tn. Ali Jafar,Laki-laki,09/05/1988,Jl. Gerilya 08/01 Tanjung,088211456394,14/02/2023,"Nyeri tenggorokan, sulit menelan",Faringitis Ringan,"Paracetamol, Lozenges",35000.0,0,36,0,172.0


LANGKAH 3 PEMERIKSAAN MISSING VALUE

In [4]:
print("=== JUMLAH MISSING VALUE TIAP KOLOM ===")
print(df.isnull().sum())

print("\nTotal missing keseluruhan:", df.isnull().sum().sum())


=== JUMLAH MISSING VALUE TIAP KOLOM ===
No                            0
Nama Pasien                   0
Jenis Kelamin                 0
Tanggal Lahir                 0
Alamat                        1
Nomor HP                      0
Tanggal Pemeriksaan           0
Pemeriksaan                   0
Diagnosa/Hasil Pemeriksaan    3
Pengobatan/Resep              9
Total Harga Pembayaran        1
Tekanan Darah                 0
Suhu                          0
Berat Badan                   0
Tinggi Badan                  0
dtype: int64

Total missing keseluruhan: 14


LANGKAH 4 CLEANING DATA

In [5]:
print("Sebelum cleaning:", df.shape)

df = df.dropna(subset=["Diagnosa/Hasil Pemeriksaan", "Pengobatan/Resep"])

print("Setelah hapus diagnosa & resep kosong:", df.shape)


Sebelum cleaning: (339, 15)
Setelah hapus diagnosa & resep kosong: (327, 15)


LANGKAH 5 FEATURE ENGINEERING (PEMISAHAN TEKANAN DARAH)

In [6]:
# Pisahkan tekanan darah menjadi dua kolom
df[["Sistolik", "Diastolik"]] = df["Tekanan Darah"].str.split("/", expand=True)

print("Contoh hasil pemisahan:")
df[["Tekanan Darah", "Sistolik", "Diastolik"]].head()


Contoh hasil pemisahan:


,Tekanan Darah,Sistolik,Diastolik
0,100/80,100,80
1,100/79,100,79
2,130/80,130,80
3,0,NaN,NaN
4,0,NaN,NaN


LANGKAH 6 PEMBERSIHAN TEKANAN DARAH TIDAK VALID

In [7]:
print("Sebelum hapus tekanan darah tidak valid:", df.shape)

df = df.dropna(subset=["Sistolik", "Diastolik"])

print("Setelah hapus tekanan darah tidak valid:", df.shape)


Sebelum hapus tekanan darah tidak valid: (327, 17)
Setelah hapus tekanan darah tidak valid: (281, 17)


LANGKAH 7 STANDARISASI NAMA KOLOM

In [8]:
df.columns = df.columns.str.lower().str.strip()

print("Daftar kolom:")
print(df.columns)


Daftar kolom:
Index(['no', 'nama pasien', 'jenis kelamin', 'tanggal lahir', 'alamat',
       'nomor hp', 'tanggal pemeriksaan', 'pemeriksaan',
       'diagnosa/hasil pemeriksaan', 'pengobatan/resep',
       'total harga pembayaran', 'tekanan darah', 'suhu', 'berat badan',
       'tinggi badan', 'sistolik', 'diastolik'],
      dtype='object')


KONVERSI KE NUMERIK

In [9]:
df["sistolik"] = pd.to_numeric(df["sistolik"], errors="coerce")
df["diastolik"] = pd.to_numeric(df["diastolik"], errors="coerce")
df["suhu"] = pd.to_numeric(df["suhu"], errors="coerce")
df["berat badan"] = pd.to_numeric(df["berat badan"], errors="coerce")
df["tinggi badan"] = pd.to_numeric(df["tinggi badan"], errors="coerce")

print("Jumlah NaN setelah konversi:")
print(df[["sistolik","diastolik","suhu","berat badan","tinggi badan"]].isnull().sum())


Jumlah NaN setelah konversi:
sistolik        0
diastolik       0
suhu            0
berat badan     1
tinggi badan    0
dtype: int64


LANGKAH 9 REPLACE 0 JADI NaN DAN IMPUTASI

In [10]:
df["suhu"] = df["suhu"].replace(0, np.nan)
df["berat badan"] = df["berat badan"].replace(0, np.nan)
df["tinggi badan"] = df["tinggi badan"].replace(0, np.nan)

df["suhu"] = df["suhu"].fillna(df["suhu"].median())
df["berat badan"] = df["berat badan"].fillna(df["berat badan"].median())
df["tinggi badan"] = df["tinggi badan"].fillna(df["tinggi badan"].median())

print("Sisa missing value:")
print(df.isnull().sum())

print("Jumlah data akhir tahap 1:", df.shape)


Sisa missing value:
no                            0
nama pasien                   0
jenis kelamin                 0
tanggal lahir                 0
alamat                        1
nomor hp                      0
tanggal pemeriksaan           0
pemeriksaan                   0
diagnosa/hasil pemeriksaan    0
pengobatan/resep              0
total harga pembayaran        1
tekanan darah                 0
suhu                          0
berat badan                   0
tinggi badan                  0
sistolik                      0
diastolik                     0
dtype: int64
Jumlah data akhir tahap 1: (281, 17)


**TAHAP 2 ENCODING VARIABEL FITUR**

LANGKAH 1 ENCODING JENIS KELAMIN

In [11]:
# ==============================
# Encoding Jenis Kelamin
# ==============================

print("Nilai unik sebelum encoding:")
print(df["jenis kelamin"].unique())

# Standarisasi teks
df["jenis kelamin"] = df["jenis kelamin"].str.lower().str.strip()

# Mapping manual agar jelas untuk TA
df["jenis kelamin"] = df["jenis kelamin"].map({
    "laki-laki": 0,
    "perempuan": 1
})

print("\nDistribusi setelah encoding:")
print(df["jenis kelamin"].value_counts())

print("\nTipe data:")
print(df["jenis kelamin"].dtype)


Nilai unik sebelum encoding:
['Laki-laki' 'Perempuan']

Distribusi setelah encoding:
jenis kelamin
0    151
1    130
Name: count, dtype: int64

Tipe data:
int64


LANGKAH 2 ENCODING TARGET

In [12]:
print(df.columns)


Index(['no', 'nama pasien', 'jenis kelamin', 'tanggal lahir', 'alamat',
       'nomor hp', 'tanggal pemeriksaan', 'pemeriksaan',
       'diagnosa/hasil pemeriksaan', 'pengobatan/resep',
       'total harga pembayaran', 'tekanan darah', 'suhu', 'berat badan',
       'tinggi badan', 'sistolik', 'diastolik'],
      dtype='object')


In [13]:
def kategori_penyakit(diagnosa):
    d = str(diagnosa).lower()

    if "ispa" in d or "cold" in d or "faringitis" in d:
        return "Infeksi"

    elif "gastritis" in d or "diare" in d:
        return "Gangguan_Pencernaan"

    elif "cephalgia" in d or "myalgia" in d:
        return "Nyeri_Ringan"

    elif "dermatitis" in d:
        return "Gangguan_Kulit"

    else:
        return "Non_Spesifik"

df["kategori_penyakit"] = df["diagnosa/hasil pemeriksaan"].apply(kategori_penyakit)

print(df["kategori_penyakit"].value_counts())


kategori_penyakit
Non_Spesifik           91
Infeksi                68
Gangguan_Pencernaan    51
Nyeri_Ringan           50
Gangguan_Kulit         21
Name: count, dtype: int64


In [14]:
# ==============================
# Gabungkan menjadi 2 kelas
# ==============================

def kategori_2_kelas(x):
    if x == "Non_Spesifik":
        return "Keluhan_Non_Spesifik"
    else:
        return "Penyakit_Spesifik"

df["kategori_final"] = df["kategori_penyakit"].apply(kategori_2_kelas)

print(df["kategori_final"].value_counts())


kategori_final
Penyakit_Spesifik       190
Keluhan_Non_Spesifik     91
Name: count, dtype: int64


LANGKAH 3 PEMISAHAN FITUR DAN TARGET

In [15]:
# =====================================
# ENCODING TARGET (Kategori Penyakit)
# =====================================

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df["target"] = le.fit_transform(df["kategori_final"])

print("Mapping target:")
for i, label in enumerate(le.classes_):
    print(i, "=", label)

print(df["target"].value_counts())



Mapping target:
0 = Keluhan_Non_Spesifik
1 = Penyakit_Spesifik
target
1    190
0     91
Name: count, dtype: int64


In [17]:
from sklearn.preprocessing import LabelEncoder

le_target = LabelEncoder()

df["target_encoded"] = le_target.fit_transform(df["kategori_penyakit"])

print("Mapping Label Encoding:")
for i, label in enumerate(le_target.classes_):
    print(i, "=", label)

print("\nContoh hasil encoding:")
print(df[["kategori_penyakit", "target_encoded"]].head())


Mapping Label Encoding:
0 = Gangguan_Kulit
1 = Gangguan_Pencernaan
2 = Infeksi
3 = Non_Spesifik
4 = Nyeri_Ringan

Contoh hasil encoding:
  kategori_penyakit  target_encoded
0      Non_Spesifik               3
1      Non_Spesifik               3
2           Infeksi               2
5      Non_Spesifik               3
7      Non_Spesifik               3


In [18]:
# ==============================
# Pemisahan Fitur dan Target
# ==============================

X = df[[
    "jenis kelamin",
    "suhu",
    "berat badan",
    "tinggi badan",
    "sistolik",
    "diastolik"
]]

y = df["target_encoded"]

print("Jumlah fitur (X):", X.shape)
print("Jumlah target (y):", y.shape)

print("\nContoh data fitur:")
print(X.head())


Jumlah fitur (X): (281, 6)
Jumlah target (y): (281,)

Contoh data fitur:
   jenis kelamin  suhu  berat badan  tinggi badan  sistolik  diastolik
0              0  36.0         63.0         163.5       100         80
1              1  36.0         64.0         163.5       100         79
2              0  38.0         59.5         163.5       130         80
5              0  36.0         67.0         163.5       140         80
7              1  36.0         64.0         163.5       110         70


**TAHAP 3 PEMODELAN RANDOM FOREST**

LANGKAH 1 SPLIT DATA TRAINING DAN TESTING

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Data training:", X_train.shape)
print("Data testing:", X_test.shape)


Data training: (224, 6)
Data testing: (57, 6)


In [21]:
X = df[["jenis kelamin", "suhu", "berat badan",
        "tinggi badan", "sistolik", "diastolik"]]

y = df["target"]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Data training:", X_train.shape)
print("Data testing:", X_test.shape)


Data training: (224, 6)
Data testing: (57, 6)


In [23]:
print("Distribusi y_train:")
print(y_train.value_counts())

print("\nDistribusi y_test:")
print(y_test.value_counts())


Distribusi y_train:
target
1    151
0     73
Name: count, dtype: int64

Distribusi y_test:
target
1    39
0    18
Name: count, dtype: int64


LANGKAH 2 TRAINING MODEL RANDOM FOREST

In [24]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight="balanced"
)

rf.fit(X_train, y_train)

print("Model Random Forest berhasil dilatih.")


Model Random Forest berhasil dilatih.


In [25]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    class_weight="balanced"
)

rf.fit(X_train, y_train)

print("Model Random Forest final berhasil dilatih.")


Model Random Forest final berhasil dilatih.


LANGKAH 3 EVALUASI MODEL

In [26]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.9122807017543859

Classification Report:
              precision    recall  f1-score   support

           0       0.78      1.00      0.88        18
           1       1.00      0.87      0.93        39

    accuracy                           0.91        57
   macro avg       0.89      0.94      0.90        57
weighted avg       0.93      0.91      0.91        57


Confusion Matrix:
[[18  0]
 [ 5 34]]


**TAHAP 4 SISTEM REKOMENDASI OBAT**

LANGKAH 1 BUAT MAPPING KATEGORI -> OBAT TERBANYAK

In [27]:
# ================================
# Mapping kategori → obat terbanyak
# ================================

rekomendasi_obat = (
    df.groupby("kategori_final")["pengobatan/resep"]
    .agg(lambda x: x.value_counts().index[0])
)

print("Mapping Rekomendasi Obat:")
print(rekomendasi_obat)


Mapping Rekomendasi Obat:
kategori_final
Keluhan_Non_Spesifik                         0
Penyakit_Spesifik       Paracetamol, Ibuprofen
Name: pengobatan/resep, dtype: object


In [28]:
# Bersihkan resep yang bernilai 0 atau kosong
df["pengobatan/resep"] = df["pengobatan/resep"].replace(0, np.nan)
df["pengobatan/resep"] = df["pengobatan/resep"].replace("-", np.nan)

df = df.dropna(subset=["pengobatan/resep"])

# Buat ulang mapping
rekomendasi_obat = (
    df.groupby("kategori_final")["pengobatan/resep"]
    .agg(lambda x: x.value_counts().index[0])
)

print("Mapping Rekomendasi Obat (Sudah Bersih):")
print(rekomendasi_obat)


Mapping Rekomendasi Obat (Sudah Bersih):
kategori_final
Keluhan_Non_Spesifik    Asam tranexsamat 2x1 tablet, Vit C 3x2 tablet
Penyakit_Spesifik                              Paracetamol, Ibuprofen
Name: pengobatan/resep, dtype: object


LANGKAH 2 MEMBUAT FUNGSI SISTEM REKOMENDASI

In [29]:
# ==========================================
# FUNGSI SISTEM REKOMENDASI OBAT OTOMATIS
# ==========================================

def rekomendasi_pasien(jenis_kelamin, suhu, berat, tinggi, sistolik, diastolik):

    # Buat DataFrame 1 baris seperti data training
    data_input = pd.DataFrame([{
        "jenis kelamin": jenis_kelamin,
        "suhu": suhu,
        "berat badan": berat,
        "tinggi badan": tinggi,
        "sistolik": sistolik,
        "diastolik": diastolik
    }])

    # Prediksi kategori
    pred = rf.predict(data_input)[0]

    # Ubah ke label asli
    kategori_pred = le.inverse_transform([pred])[0]

    # Ambil rekomendasi obat
    obat_rekom = rekomendasi_obat[kategori_pred]

    print("=== HASIL SISTEM REKOMENDASI ===")
    print("Kategori Prediksi :", kategori_pred)
    print("Rekomendasi Obat  :", obat_rekom)


In [30]:
rekomendasi_pasien(
    jenis_kelamin=0,   # laki-laki
    suhu=38,
    berat=60,
    tinggi=165,
    sistolik=120,
    diastolik=80
)


=== HASIL SISTEM REKOMENDASI ===
Kategori Prediksi : Penyakit_Spesifik
Rekomendasi Obat  : Paracetamol, Ibuprofen
